# SIRS with mobility and network structure: SIMULATION SF vs ER

We fixed the epidemic parameters in a "critical" point at which most of the times the epidemics does not survive and dies out after some iterations. The topology of the two networks is also kept fixed. We now explore the role of the mobility parameter p_mob. In this case simulation is done coupling a scale free network and an Erdos-Renyi network.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx
import os
import pandas as pd
import time 
# to install tqdm
#! pip install tqdm
from tqdm import tnrange, trange

# import custom module
import SIRS

In [ ]:
# needed only if you made some changes at SIRS.py and want to import the updated version of the module
from importlib import reload
reload(SIRS)

In [ ]:
#mobility and topology parameters
N = 1000
I_sf_init = 5
I_er_init = 0
mean_degree = 4

In [ ]:
beta0 = 0.3
gamma = 0.016
mu = 0.15

#mobility parameters
p_mob_list = np.linspace(0.03,0.2,20)
#p_mob_list = np.concatenate([[0],p_mob_list])
print('Number of travellers:')
for p in p_mob_list:
    print('p=%.2f    N=%d'%(p, int(p*N)))

In [ ]:
beta = beta0/mean_degree
infection_params = dict(beta=beta, mu=mu, gamma=gamma)


UP_directory_name = 'data_SF_ER/'+str(beta)+"_"+str(mu)+"_"+str(gamma) 
os.system("mkdir data_SF_ER")
os.system("mkdir "+UP_directory_name)
start = time.time()

for j, p_mob in enumerate(p_mob_list):

    print('Simulation %d/%d'%(j+1,len(p_mob_list)), end =' - ')
    print("P_mob: %.3f"% p_mob)

    #saving stuff for future analysis
    directory_name = UP_directory_name+"/pmob_"+"{:3f}".format(p_mob)
    os.system("mkdir "+directory_name)

    #saving epidemics, topology and mobility parameters

    params=dict(N=N, I_sf_init=I_sf_init, I_er_nit=I_er_init, p_mob=p_mob, mean_degree=mean_degree,
                eps=eps, **infection_params)

    parameters = pd.DataFrame(params, index=[0])
    save=directory_name+"/parameters.csv"
    parameters.to_csv(save)



    #STARTING SIMULATION
    n_runs = 100
    n_iter = 1000



    for run in tnrange(n_runs, desc='1st loop', leave=True):


        # prepare systems
        state_sf_init, state_er_init, variables_net_sf, variables_net_er = SIRS.prepare_two_sys(N, I_sf_init, I_er_init, 
                                                                                      p_mob, mean_degree)

        #saving initial states
        save = directory_name + "/state_sf_init_"+str(run)+".txt"
        np.savetxt(save, state_sf_init.astype(int), fmt ='%d')

        save = directory_name + "/state_er_init_"+str(run)+".txt"
        np.savetxt(save, state_sf_init.astype(int), fmt ='%d')

        save = directory_name + "/travellers_sf_"+str(run)+".txt"
        travellers_sf = variables_net_sf["travellers_sf"]
        np.savetxt(save, travellers_sf.astype(int), fmt ='%d')
        
        save = directory_name + "/travellers_er_"+str(run)+".txt"
        travellers_er = variables_net_er["travellers_er"]
        np.savetxt(save, travellers_er.astype(int), fmt ='%d')
        
        #saving initial variables: we just save the IDs of the travelling nodes and their degrees
        save = directory_name + "/deg_trav_sf_"+str(run)+".txt"
        deg_sf = variables_net_sf["A_sf"].sum(axis=1)
        deg_trav_sf = deg_sf[travellers_sf]
        np.savetxt(save, deg_trav_sf.astype(int), fmt ='%d')
        
        save = directory_name + "/deg_trav_er_"+str(run)+".txt"
        deg_er = variables_net_er["A_er"].sum(axis=1)
        deg_trav_er = deg_er[travellers_er]
        np.savetxt(save, deg_trav_er.astype(int), fmt ='%d')

        state_sf = np.copy(state_sf_init)
        state_er = np.copy(state_er_init)

        S_sf = [N - I_sf_init]
        I_sf = [I_sf_init]
        R_sf = [0]
        S_er = [N - I_er_init]
        I_er = [I_er_init]
        R_er = [0]
        t_vec = []
        for i in tnrange(n_iter, desc='2nd loop', leave=False):
            state_sf, state_er = SIRS.two_sys_full_SIRS_step(state_sf, state_er, **variables_net_sf, 
                                                    **variables_net_er, **infection_params)
            S_sf.append(state_sf[:,0].sum())
            I_sf.append(state_sf[:,1].sum())
            R_sf.append(state_sf[:,2].sum())
            S_er.append(state_er[:,0].sum())
            I_er.append(state_er[:,1].sum())
            R_er.append(state_er[:,2].sum())
            #t_vec.append(time.time()-start)

        #tot_time = time.time()-start
        #print("Total time elapsed: %.2f s"%tot_time)
        #print("Time per iteration: %.4f s"%(tot_time/n_iter))

        S_sf = np.array(S_sf)
        I_sf = np.array(I_sf)
        R_sf = np.array(R_sf)
        S_er = np.array(S_er)
        I_er = np.array(I_er)
        R_er = np.array(R_er)
        t_vec = np.array(t_vec)

        #save timeseries of SIR
        SIR_sf = np.vstack((S_sf, I_sf, R_sf)).T
        save = directory_name+"/SIR_sf_"+str(run)+".txt"
        np.savetxt(save, SIR_sf.astype(int), fmt ='%d')
        SIR_er = np.vstack((S_er, I_er, R_er)).T
        save = directory_name+"/SIR_er_"+str(run)+".txt"
        np.savetxt(save, SIR_er.astype(int), fmt ='%d')

    
    dt = (time.time() - start)/3600
    # estimated time of arrival
    eta = (dt/(j+1))*(len(p_mob_list)-j-1)
    print("Total time elapsed: %.2f - ETA: %.2f"%(dt, eta))